In [1]:
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

In [2]:
""" pdf_loader = DirectoryLoader('../../../../data/tenders', glob="**/*.pdf")
txt_loader = DirectoryLoader('../../../../data/tenders', glob="**/*.txt")
word_loader = DirectoryLoader('../../../../data/tenders', glob="**/*.docx")
csv_loader = DirectoryLoader('../../../../data/tenders', glob="**/*.csv") """

' pdf_loader = DirectoryLoader(\'../../../../data/tenders\', glob="**/*.pdf")\ntxt_loader = DirectoryLoader(\'../../../../data/tenders\', glob="**/*.txt")\nword_loader = DirectoryLoader(\'../../../../data/tenders\', glob="**/*.docx")\ncsv_loader = DirectoryLoader(\'../../../../data/tenders\', glob="**/*.csv") '

In [3]:
from bs4 import BeautifulSoup as Soup

url = "https://developers.hubspot.com/docs/"
recursive_url_loader = RecursiveUrlLoader(url=url, max_depth=200, extractor=lambda x: Soup(x, "html.parser").text)

loaders = [recursive_url_loader]
documents = []
for loader in loaders:
    documents.extend(loader.load())

print(f"Total number of documents: {len(documents)}")

Unable to load from https://developers.hubspot.com/docs/api/crm/extensions/crm-cards. Received error HTTPSConnectionPool(host='developers.hubspot.com', port=443): Read timed out. (read timeout=10) of type ReadTimeout
Unable to load from https://developers.hubspot.com/docs/methods/comments/delete_comments_comment_id. Received error HTTPSConnectionPool(host='developers.hubspot.com', port=443): Read timed out. (read timeout=10) of type ReadTimeout
Unable to load from https://developers.hubspot.com/docs/methods/files/post_folders. Received error HTTPSConnectionPool(host='legacydocs.hubspot.com', port=443): Read timed out. of type ConnectionError
Unable to load from https://developers.hubspot.com/docs/methods/pages/post_pages_page_id_publish_action. Received error HTTPSConnectionPool(host='developers.hubspot.com', port=443): Max retries exceeded with url: /docs/methods/pages/post_pages_page_id_publish_action (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd0

Total number of documents: 661


In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
huggingfacehub_api_key= os.getenv('HUGGINGFACEHUB_API_TOKEN')
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
texts = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings()

Created a chunk of size 1825, which is longer than the specified 1000
Created a chunk of size 1019, which is longer than the specified 1000
Created a chunk of size 1204, which is longer than the specified 1000
Created a chunk of size 1153, which is longer than the specified 1000
Created a chunk of size 1244, which is longer than the specified 1000
Created a chunk of size 1722, which is longer than the specified 1000
Created a chunk of size 1169, which is longer than the specified 1000
Created a chunk of size 1579, which is longer than the specified 1000
Created a chunk of size 1009, which is longer than the specified 1000
Created a chunk of size 1565, which is longer than the specified 1000
Created a chunk of size 1185, which is longer than the specified 1000
Created a chunk of size 1151, which is longer than the specified 1000
Created a chunk of size 1200, which is longer than the specified 1000
Created a chunk of size 1078, which is longer than the specified 1000
Created a chunk of s

In [6]:
db = Chroma.from_documents(texts, embeddings, collection_name="tenders")

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()
google_api_key= os.getenv('GOOGLE_API_KEY')

In [8]:
import os
from dotenv import load_dotenv

load_dotenv()
cohere_api_key= os.getenv('COHERE_API_KEY')

In [9]:
from langchain.chat_models import ChatLiteLLM
cohere_chat_llm = ChatLiteLLM(model="command-nightly")

In [10]:
from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm

In [11]:
google_palm_llm = GooglePalm()
google_palm_llm.temperature = 0.75

NotImplementedError: Need to determine which default deprecation schedule to use. within ?? minor releases

In [ ]:
tenders_docs_chain = RetrievalQA.from_chain_type(
    llm=cohere_chat_llm, chain_type="stuff", retriever=db.as_retriever()
)

In [ ]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.utilities import PythonREPL
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import DuckDuckGoSearchRun 
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.tools import ShellTool
from langchain.agents import load_tools

In [ ]:
#DuckDuckGo Class
duckduck_search = DuckDuckGoSearchRun()
shell_tool = ShellTool()


#Python REPL Class
python_repl = PythonREPL()

shell_tool = Tool(
    name="shell",
    description="This tool gives you access to a system's shell. Can be used to run shell commands",
    func=shell_tool.run
)


#DuckDuckGo Tool
duckduck_tool=Tool(
        name = "DuckDuckGo",
        func=duckduck_search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
)

#Python REPL Agent Tool Class
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run
)

ImportError: BashProcess has been moved to langchain experimental.To use this tool, install langchain-experimental with `pip install langchain-experimental`.

: 

In [ ]:
tools = [
    Tool(
        name="Tender Docs Chain",
        func=tenders_docs_chain.run,
        description="Useful for when you need to answer questions about Tenders. output should be code",
    ),
    Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run
    ), 
    Tool(
        name = "DuckDuckGo",
        func=duckduck_search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ), 
    Tool(
    name="shell",
    description="This tool gives you access to a system's shell. Can be used to run shell commands",
    func=shell_tool.run
    )
]

In [ ]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, google_palm_llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.run(
    """You're a HubSpot Docs bot with the experience of a seasoned developer. How I can conenct an external chatbot with OpenAI's API"""
)



> Entering new AgentExecutor chain...
I need to use the Tender Docs Chain to answer this question
Action: Tender Docs Chain
Action Input: 

Your text contains a trailing whitespace, which has been trimmed to ensure high quality generations.



Observation: Sorry, I am unable to provide any answers pertaining to the Acceptable Use Policy. Is there anything else I could help you with today? 
Thought:I don't know the final answer
Final Answer: unanswerable

> Finished chain.


'unanswerable'

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate


In [ ]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(name="source", description="source used to answer the user's question, should be a website.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("answer the users question as best as possible.\n{format_instructions}\n{question}")  
    ],
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
_input = prompt.format_prompt(question="what's a quaternion?")
output = google_palm_llm(_input.to_string())

NameError: name 'google_palm_llm' is not defined

In [ ]:
output_parser.parse(output)

NameError: name 'output' is not defined